In [12]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
import xml.etree.ElementTree as xet
import xml.etree.ElementTree as ET
from PIL import Image
import os
import xml.etree.ElementTree as ET
import shutil

In [16]:
# Defines constants
dataset_path = "data"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")
yolo_labels_path = os.path.join(dataset_path, "yolo_labels")

# Ensure YOLO labels directory exists
os.makedirs(yolo_labels_path, exist_ok=True)

In [ ]:
# Convert XML labels to YOLO format
def convert_to_yolo(xml_file, output_dir, image_file):
    # Get image dimensions
    with Image.open(image_file) as img:
        width, height = img.size

    # Parse XML label
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_data = []

    for obj in root.findall("object"):
        class_name = obj.find("name").text
        class_id = 0  # Replace with actual class mapping if needed
        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)

        # Normalize coordinates
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height

        yolo_data.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

    # Save YOLO label
    output_file = os.path.join(output_dir, os.path.basename(xml_file).replace(".xml", ".txt"))
    with open(output_file, "w") as f:
        f.write("\n".join(yolo_data))

# Process all XML files
for xml_file in os.listdir(labels_path):
    if xml_file.endswith(".xml"):
        image_file = os.path.join(images_path, xml_file.replace(".xml", ".png"))
        if os.path.exists(image_file):
            convert_to_yolo(os.path.join(labels_path, xml_file), yolo_labels_path, image_file)

# Split Dataset into YOLO Training and Validation Format
# Paths for split
images_train_path = os.path.join(images_path, "train")
images_val_path = os.path.join(images_path, "val")
labels_train_path = os.path.join(labels_path, "train")
labels_val_path = os.path.join(labels_path, "val")

os.makedirs(images_train_path, exist_ok=True)
os.makedirs(images_val_path, exist_ok=True)
os.makedirs(labels_train_path, exist_ok=True)
os.makedirs(labels_val_path, exist_ok=True)

# List all images and corresponding YOLO label files
images = [f for f in os.listdir(images_path) if f.endswith(".png")]
labels = [f.replace(".png", ".txt") for f in images]

print(labels)
print(images)

# Split into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# Move training data
for img, lbl in zip(train_images, train_labels):
    shutil.move(os.path.join(images_path, img), os.path.join(images_train_path, img))
    shutil.move(os.path.join(yolo_labels_path, lbl), os.path.join(labels_train_path, lbl))

# Move validation data
for img, lbl in zip(val_images, val_labels):
    shutil.move(os.path.join(images_path, img), os.path.join(images_val_path, img))
    shutil.move(os.path.join(yolo_labels_path, lbl), os.path.join(labels_val_path, lbl))

print("Dataset successfully converted and split!")


[]


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.